In [2]:
import os
import sys
import shutil

sys.path.append("../../.github/scripts/")
from utils.plantuml_format import PlantUMLFormatter, \
    plantUML_format_dictionary

from utils.json_manipulation import JSONManipulationFormatter, \
    conditions_cardinality, \
    conditions_controlled_vocabulary, \
    conditions_ontology_validation

# Creation of PlantUML syntax JSONs

In [3]:
# We simply run the whole cell changing the are_schemas from False to True.
#   Change any other custom option if desired.
are_schemas = False

if are_schemas:
    input_dir = "../../schemas/"
    output_dir = "../../docs/json_diagrams/json_schemas/"
    resolve_refs = True
else:
    input_dir = "../../examples/json_validation_tests/"
    output_dir = "../../docs/json_diagrams/json_documents/"
    resolve_refs = False

reduced_depth = 2

for file in os.scandir(input_dir):
    if file.name.endswith(".json") and file.is_file():
        filepath = file.path
        object_type = file.name.split('_', 1)[0]        # e.g. ("assay" + _ + "valid-1.json")
        custom_output_dir = output_dir + object_type
        if not os.path.exists(custom_output_dir):
            os.makedirs(custom_output_dir)

        # We initiate the JSON manipulation class with the file
        formatter = JSONManipulationFormatter(json_filepath=filepath)
        
        # - #
        # The whole JSON object, just slightly formatted and with no highlighting
        # - #
        if resolve_refs:
            formatter.resolve_json_references()
        formatter.add_newlines()

        header = PlantUMLFormatter(json_data=formatter.current_json)

        # We save the PUML file
        type_tag = "whole"
        output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
        header.save_all(output_filepath=output_filepath, overwrite=True)

        if are_schemas:
            # - #
            # The whole JSON object but without resolved references
            # - #
            formatter.restart_json()
            formatter.add_newlines()

            header = PlantUMLFormatter(json_data=formatter.current_json)

            # We save the PUML file
            type_tag = "wholeUnresolved"
            output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
            header.save_all(output_filepath=output_filepath, overwrite=True)
        
        if not are_schemas:
            # - #
            # Only the ones with "objectId" or relationship objects, to show cardinality between objects
            # - #
            formatter.restart_json()
            if resolve_refs:
                formatter.resolve_json_references()
            formatter.subset_json(condition_dict=conditions_cardinality) # We subset based on cardinality
            formatter.add_newlines()

            header = PlantUMLFormatter(json_data=formatter.current_json)

            # We save the PUML file
            type_tag = "cardinality"
            output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
            header.save_all(output_filepath=output_filepath, overwrite=True)

        if are_schemas:
            # - #
            # The whole JSON object, but with highlighted "enum" in properties (controlled vocabularies)
            # - # 
            formatter.restart_json()
            if resolve_refs:
                formatter.resolve_json_references()
            formatter.add_newlines()

            header = PlantUMLFormatter(json_data=formatter.current_json)
            header.add_jsonpaths(condition_dict=conditions_controlled_vocabulary) # We highlight based on the "enum"
            header.format_json_paths(plantUML_format_dictionary=plantUML_format_dictionary) 
            header.create_highlights_header()

            # We save the PUML file
            type_tag = "CVhighlights"
            output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
            header.save_all(output_filepath=output_filepath, overwrite=True)

            # - #
            # Subset of the JSON object with "enum" as properties (controlled vocabularies)
            # - # 
            formatter.restart_json()
            if resolve_refs:
                formatter.resolve_json_references()
            formatter.subset_json(condition_dict=conditions_controlled_vocabulary) # Different
            formatter.add_newlines()

            header = PlantUMLFormatter(json_data=formatter.current_json)

            # We save the PUML file
            type_tag = "CVsubset"
            output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
            header.save_all(output_filepath=output_filepath, overwrite=True)


        if not are_schemas:
            # - #
            # The whole JSON object, but with highlighted "termId" in properties (ontology validation)
            # - #
            formatter.restart_json()
            if resolve_refs:
                formatter.resolve_json_references()
            formatter.add_newlines()

            header = PlantUMLFormatter(json_data=formatter.current_json)
            header.add_jsonpaths(condition_dict=conditions_ontology_validation) # We highlight based on the "termId"
            header.format_json_paths(plantUML_format_dictionary=plantUML_format_dictionary) 
            header.create_highlights_header()

            # We save the PUML file
            type_tag = "ontologiesHighlight"
            output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
            header.save_all(output_filepath=output_filepath, overwrite=True)

            # - #
            # Subset of the JSON object with "termId" in the properties (ontology validation)
            # - #
            formatter.restart_json()
            if resolve_refs:
                formatter.resolve_json_references()
            formatter.subset_json(condition_dict=conditions_ontology_validation) # we subset based on "termId"
            formatter.add_newlines()

            header = PlantUMLFormatter(json_data=formatter.current_json)

            # We save the PUML file
            type_tag = "ontologiesSubset"
            output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
            header.save_all(output_filepath=output_filepath, overwrite=True)

        # - #
        # A JSON with only 2 levels, but at the same time having highlighted properties whose leaves have CURIEs
        # - #
        formatter.restart_json()
        header = PlantUMLFormatter(json_data=formatter.current_json)
        header.add_jsonpaths(condition_dict=conditions_ontology_validation) # We highlight based on the "termId"
        header.format_json_paths(plantUML_format_dictionary=plantUML_format_dictionary) 
        header.create_highlights_header()   # We use the bigger JSON to create the highlights

        formatter.reduce_depth(reduced_depth)   # We create the smaller JSON
        formatter.add_newlines()
        header.load_json_data(json_data=formatter.current_json) # We give it the smaller JSON now

        # We save the PUML file
        type_tag = "small"
        output_filepath = custom_output_dir + "/" + os.path.splitext(file.name)[0] + "_" + type_tag + ".puml"
        header.save_all(output_filepath=output_filepath, overwrite=True)

# Creation of SVGs based on JSONs
- Take into account that you will need a file ``plantuml.jar`` (see [documentation](https://plantuml.com/command-line) and [downloads](https://plantuml.com/download)), in our case ``plantuml-1.2023.1.jar``, in the same directory as this notebook

In [5]:
plantuml_filepath = "plantuml-1.2023.1.jar"
assert os.path.isfile(plantuml_filepath)

for root, dirs, files in os.walk(output_dir):
    # Loop through all files in the output directory
    for file_name in files:
        if file_name.endswith('.puml'):
            print("- Current filename: ", file_name)
            # Get the full path to the file
            file_path = os.path.join(root, file_name)
            output_dir_svg = os.path.join('../..', root)

            
            # Create the SVGs using PlantUML
            !java -jar {plantuml_filepath} -tsvg -o {output_dir_svg} {file_path}
            
            # Create the puml_files directory if it doesn't exist
            puml_dir = os.path.join(root, 'puml_files')
            if not os.path.exists(puml_dir):
                os.mkdir(puml_dir)
                
            # Move the file to the puml_files directory
            dest_path = os.path.join(puml_dir, file_name)
            shutil.move(file_path, dest_path)

- Current filename:  analysis_valid-1_cardinality.puml
- Current filename:  analysis_valid-1_ontologiesHighlight.puml
- Current filename:  analysis_valid-1_ontologiesSubset.puml
- Current filename:  analysis_valid-1_small.puml
- Current filename:  analysis_valid-1_whole.puml
- Current filename:  assay_valid-1_array_cardinality.puml
- Current filename:  assay_valid-1_array_ontologiesHighlight.puml
- Current filename:  assay_valid-1_array_ontologiesSubset.puml
- Current filename:  assay_valid-1_array_small.puml
- Current filename:  assay_valid-1_array_whole.puml
- Current filename:  assay_valid-2_sequencing_cardinality.puml
- Current filename:  assay_valid-2_sequencing_ontologiesHighlight.puml
- Current filename:  assay_valid-2_sequencing_ontologiesSubset.puml
- Current filename:  assay_valid-2_sequencing_small.puml
- Current filename:  assay_valid-2_sequencing_whole.puml
- Current filename:  DAC_valid-1_cardinality.puml
- Current filename:  DAC_valid-1_ontologiesHighlight.puml
- Current

Exception in thread "main" java.lang.UnsupportedOperationException: 3vnixbvvmty9ydvf0l1929gle
	at smetana.core.Macro.UNSUPPORTED(Macro.java:140)
	at gen.lib.common.shapes__c.parse_reclbl(shapes__c.java:1563)
	at gen.lib.common.shapes__c.record_init(shapes__c.java:1805)
	at gen.lib.common.shapes__c$7.exe(shapes__c.java:1778)
	at gen.lib.common.utils__c.common_init_node(utils__c.java:644)
	at gen.lib.dotgen.dotinit__c.dot_init_node(dotinit__c.java:156)
	at gen.lib.dotgen.dotinit__c.dot_init_node_edge(dotinit__c.java:214)
	at gen.lib.dotgen.dotinit__c.dotLayout(dotinit__c.java:357)
	at gen.lib.dotgen.dotinit__c.doDot(dotinit__c.java:492)
	at gen.lib.dotgen.dotinit__c.dot_layout(dotinit__c.java:547)
	at gen.lib.dotgen.dotinit__c$2.exe(dotinit__c.java:539)
	at gen.lib.gvc.gvlayout__c.gvLayoutJobs(gvlayout__c.java:153)
	at net.sourceforge.plantuml.jsondiagram.SmetanaForJson.initGraph(SmetanaForJson.java:190)
	at net.sourceforge.plantuml.jsondiagram.SmetanaForJson.drawMe(SmetanaForJson.java:1

- Current filename:  DAC_valid-1_whole.puml
- Current filename:  dataset_valid-1_cardinality.puml
- Current filename:  dataset_valid-1_ontologiesHighlight.puml
- Current filename:  dataset_valid-1_ontologiesSubset.puml
- Current filename:  dataset_valid-1_small.puml


Exception in thread "main" java.lang.UnsupportedOperationException: 3vnixbvvmty9ydvf0l1929gle
	at smetana.core.Macro.UNSUPPORTED(Macro.java:140)
	at gen.lib.common.shapes__c.parse_reclbl(shapes__c.java:1563)
	at gen.lib.common.shapes__c.record_init(shapes__c.java:1805)
	at gen.lib.common.shapes__c$7.exe(shapes__c.java:1778)
	at gen.lib.common.utils__c.common_init_node(utils__c.java:644)
	at gen.lib.dotgen.dotinit__c.dot_init_node(dotinit__c.java:156)
	at gen.lib.dotgen.dotinit__c.dot_init_node_edge(dotinit__c.java:214)
	at gen.lib.dotgen.dotinit__c.dotLayout(dotinit__c.java:357)
	at gen.lib.dotgen.dotinit__c.doDot(dotinit__c.java:492)
	at gen.lib.dotgen.dotinit__c.dot_layout(dotinit__c.java:547)
	at gen.lib.dotgen.dotinit__c$2.exe(dotinit__c.java:539)
	at gen.lib.gvc.gvlayout__c.gvLayoutJobs(gvlayout__c.java:153)
	at net.sourceforge.plantuml.jsondiagram.SmetanaForJson.initGraph(SmetanaForJson.java:190)
	at net.sourceforge.plantuml.jsondiagram.SmetanaForJson.drawMe(SmetanaForJson.java:1

- Current filename:  dataset_valid-1_whole.puml
- Current filename:  experiment_valid-1_cardinality.puml
- Current filename:  experiment_valid-1_ontologiesHighlight.puml
- Current filename:  experiment_valid-1_ontologiesSubset.puml
- Current filename:  experiment_valid-1_small.puml
- Current filename:  experiment_valid-1_whole.puml
- Current filename:  individual_valid-1_cardinality.puml
- Current filename:  individual_valid-1_ontologiesHighlight.puml
- Current filename:  individual_valid-1_ontologiesSubset.puml
- Current filename:  individual_valid-1_small.puml
- Current filename:  individual_valid-1_whole.puml
- Current filename:  object-set_Covid19_cardinality.puml
- Current filename:  object-set_Covid19_ontologiesHighlight.puml
- Current filename:  object-set_Covid19_ontologiesSubset.puml
- Current filename:  object-set_Covid19_small.puml
- Current filename:  object-set_Covid19_whole.puml
- Current filename:  object-set_RareDisease_cardinality.puml
- Current filename:  object-set_R

Exception in thread "main" java.lang.UnsupportedOperationException: 3vnixbvvmty9ydvf0l1929gle
	at smetana.core.Macro.UNSUPPORTED(Macro.java:140)
	at gen.lib.common.shapes__c.parse_reclbl(shapes__c.java:1563)
	at gen.lib.common.shapes__c.record_init(shapes__c.java:1805)
	at gen.lib.common.shapes__c$7.exe(shapes__c.java:1778)
	at gen.lib.common.utils__c.common_init_node(utils__c.java:644)
	at gen.lib.dotgen.dotinit__c.dot_init_node(dotinit__c.java:156)
	at gen.lib.dotgen.dotinit__c.dot_init_node_edge(dotinit__c.java:214)
	at gen.lib.dotgen.dotinit__c.dotLayout(dotinit__c.java:357)
	at gen.lib.dotgen.dotinit__c.doDot(dotinit__c.java:492)
	at gen.lib.dotgen.dotinit__c.dot_layout(dotinit__c.java:547)
	at gen.lib.dotgen.dotinit__c$2.exe(dotinit__c.java:539)
	at gen.lib.gvc.gvlayout__c.gvLayoutJobs(gvlayout__c.java:153)
	at net.sourceforge.plantuml.jsondiagram.SmetanaForJson.initGraph(SmetanaForJson.java:190)
	at net.sourceforge.plantuml.jsondiagram.SmetanaForJson.drawMe(SmetanaForJson.java:1

- Current filename:  submission_valid-1_whole.puml
